In [2]:
import time
import os
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pyodbc
from selenium.common.exceptions import TimeoutException

# 아이디, 비밀번호를 토큰화

load_dotenv(verbose=True)

username = os.getenv('name')
password = os.getenv('Password')
database = os.getenv("db")
server = os.getenv('Server')
URL = os.getenv('URL')
driver = '{ODBC Driver 17 for SQL Server}'


# MSSQL DB 연결하여 추가해야 할 이메일 추가
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password+';Authentication=ActiveDirectoryPassword')
cursor = cnxn.cursor()
cursor.execute("SELECT email FROM Dim_Azure_User where year(start_date) = 2024") # 추가 조건
rows = cursor.fetchall()
email_list = [row.email for row in rows]


# 드라이버 셋업
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')  # Last I checked this was necessary.

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# URL 불러오기
driver.get(URL)

# 아이디 입력
time.sleep(4)
driver.find_element("name", "loginfmt").send_keys(username)
driver.find_element("id", "idSIButton9").click()

#비밀번호 입력
time.sleep(4)  # 페이지가 로드되기를 기다리는 방법
driver.find_element("name", "passwd").send_keys(password)
driver.find_element("id", "idSIButton9").click()

#로그인 유지
time.sleep(4)
driver.find_element("id", "idSIButton9").click()

# Add members 클릭
time.sleep(5)
button_locator = (By.CSS_SELECTOR, 'div[aria-label="Add members"]')
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(button_locator))
button.click()

# Frame 전환
time.sleep(5)
iframe_element = driver.find_element(By.ID, "_react_frame_2")
driver.switch_to.frame(iframe_element)


# 검색 창 찾기
search_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "SearchBox5")))
checkbox_locator = (By.XPATH, '//div[@role="checkbox" and @data-selection-toggle="true"]')

# test_li에 있는 각 검색어에 대해 검색 및 추가
for i, email in enumerate(email_list):
    search_box.send_keys(email) #검색어 입력
    time.sleep(2) # 검색 결과가 나타날 때까지 기다림
    
    try:
        checkboxes = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(checkbox_locator))
        checkboxes[0].click() #체크박스 클릭
    except TimeoutException:
        pass
        #print(f"{email}에 대한 검색 결과가 없습니다. 다음 이메일로 넘어갑니다.")
    finally:
        # 입력 값 지우기
        time.sleep(2)
        search_box.send_keys(Keys.BACKSPACE * len(email)) 
    
# # # select 하기
# # # WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "id__24"))).click()